In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, pathlib, shutil, random
import numpy as np

import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras.layers import TextVectorization
from tensorflow import keras
# from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
# import hazm as hz
from hazm import Normalizer, word_tokenize,stopwords_list,Stemmer
import re
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from tensorflow import keras
import itertools    
from keras.layers import Embedding, LSTM, Dense, Dropout,Masking
from keras.preprocessing.text import Tokenizer


In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data.csv' , on_bad_lines='skip' , delimiter='\t')
data = data[['comment' , 'label' , 'label_id']]
data.dropna(inplace=True)
data.head()

,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0


In [73]:
## استپ واچ هزم مشکل داشت و خیلی از کلماتی که نیاز داشتیم را حذف میکرد
sp="دیگران همچنان مدت چیز سایر جا طی کل کنونی بیرون مثلا کامل کاملا آنکه موارد واقعی امور اکنون بطور بخشی تحت چگونه عدم نوعی حاضر وضع مقابل کنار خویش نگاه درون زمانی بنابراین تو خیلی بزرگ خودش جز اینجا مختلف توسط نوع همچنین آنجا قبل جناح اینها طور شاید ایشان جهت طریق مانند پیدا ممکن کسانی جای کسی غیر بی قابل درباره جدید وقتی اخیر چرا بیش روی طرف جریان زیر آنچه البته فقط چیزی چون برابر هنوز بخش زمینه بین بدون استفاد همان نشان بسیاری بعد عمل روز اعلام چند آنان بلکه امروز تمام بیشتر آیا برخی علیه دیگری ویژه گذشته انجام حتی داده راه سوی ولی زمان حال تنها بسیار یعنی عنوان همین هبچ پیش وی یکی اینکه وجود شما پس چنین میان مورد چه اگر همه نه دیگر آنها باید هر او ما من تا نیز اما یک خود بر یا هم را این با آن برای و در به که از"


In [80]:
def wordprocessor(x):
  y=""
  for w in x.split():
    if len(w)>1:
      if w not in sp:
        w = re.sub(r'[،.]', r'', w)
        w= re.sub(r'([^\x00-\x7F])\1+', r'\1', w)
        y=  y + " "+ w

  return(y)
# data2=data.iloc[1:12,:]          
data['comment'] = data['comment'].apply(lambda x: wordprocessor(x))                 

In [94]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(data['comment'])
X = tokenizer.texts_to_sequences(data['comment'])
X = pad_sequences(X, maxlen=100)
y = data['label_id']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [98]:
# SARA model:
def get_model():
    model = Sequential()
    model.add(Masking(mask_value=0))
    model.add(Embedding(10000, 128, input_length=100))
    model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
model=get_model()
# Training model
model.fit(X_train, y_train, batch_size=32, epochs=2, validation_data=(X_test, y_test))

# Evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (score[1]*100))

Epoch 1/2
1737/1737 [==============================] - 444s 254ms/step - loss: 0.3768 - accuracy: 0.8345 - val_loss: 0.3618 - val_accuracy: 0.8454
Epoch 2/2
1737/1737 [==============================] - 396s 228ms/step - loss: 0.3382 - accuracy: 0.8541 - val_loss: 0.3470 - val_accuracy: 0.8475
Accuracy: 84.75%


In [ ]:
from tensorflow.keras import layers
embedding_dim = 120
max_tokens = 10000
embedding_matrix = np.zeros((max_tokens, embedding_dim))

embedding_layer = layers.Embedding(
      max_tokens,
      embedding_dim,
      embeddings_initializer=keras.initializers.Constant(embedding_matrix),
      trainable=False,
      mask_zero=True,
)
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

# model.compile(optimizer="rmsprop",                loss="binary_crossentropy",                metrics=["accuracy"])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=2, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (score[1]*100))

Epoch 1/2
1737/1737 [==============================] - 426s 215ms/step - loss: 0.6815 - accuracy: 0.5696 - val_loss: 0.6764 - val_accuracy: 0.5743
Epoch 2/2
1737/1737 [==============================] - 390s 225ms/step - loss: 0.6791 - accuracy: 0.5742 - val_loss: 0.6761 - val_accuracy: 0.5743
Accuracy: 57.43%
